In [1]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available.")

# Initialize GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  1


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Rutas a las carpetas de entrenamiento, prueba y validación para imágenes y máscaras
train_frames_dir = '../Frames/TRAIN'
train_Landmarks_dir = './Landmarks/TRAIN'
test_frames_dir = '../Frames/TEST'
test_Landmarks_dir = './Landmarks/TEST'
val_frames_dir = '../Frames/VAL'
val_Landmarks_dir = './Landmarks/VAL'

# Funciones para cargar y preprocesar imagen y máscara y convertirlo a solo un canal
def load_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image / 255.0
    return image

def load_landmarks(file_path):
    images = []
    for filename in os.listdir(file_path):
        image=cv2.imread(os.path.join(file_path, filename))
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(grayscale_image)

    stacked_image = np.stack(images, axis=-1)  # axis=-1 indica la dimensión de los canales
    stacked_image = stacked_image / 255.0
    return stacked_image

train_image_files = [os.path.join(train_frames_dir, filename) for filename in os.listdir(train_frames_dir)]
train_Landmarks_files = [os.path.join(train_Landmarks_dir, filename) for filename in os.listdir(train_Landmarks_dir)]
test_image_files = [os.path.join(test_frames_dir, filename) for filename in os.listdir(test_frames_dir)]
test_Landmarks_files = [os.path.join(test_Landmarks_dir, filename) for filename in os.listdir(test_Landmarks_dir)]
val_image_files = [os.path.join(val_frames_dir, filename) for filename in os.listdir(val_frames_dir)]
val_Landmarks_files = [os.path.join(val_Landmarks_dir, filename) for filename in os.listdir(val_Landmarks_dir)]

train_images = [load_image(file) for file in train_image_files]
train_Landmarks = [load_landmarks(file) for file in train_Landmarks_files]

test_images = [load_image(file) for file in test_image_files]
test_Landmarks = [load_landmarks(file) for file in test_Landmarks_files]

val_images = [load_image(file) for file in val_image_files]
val_Landmarks = [load_landmarks(file) for file in val_Landmarks_files]

train_images = np.array(train_images)
train_Landmarks = np.array(train_Landmarks)
test_images = np.array(test_images)
test_Landmarks = np.array(test_Landmarks)
val_images = np.array(val_images)
val_Landmarks = np.array(val_Landmarks)

KeyboardInterrupt: 

In [ ]:
# Contar que todas las imágenes y máscaras tengan el mismo tamaño
print("Train images shape: ", train_images.shape)
print("Train Landmarks shape: ", train_Landmarks.shape)
print("Test images shape: ", test_images.shape)
print("Test Landmarks shape: ", test_Landmarks.shape)
print("Validation images shape: ", val_images.shape)
print("Validation Landmarks shape: ", val_Landmarks.shape)

In [ ]:
# Agremos el path de la carpeta de modelos para poder importar el modelo
import sys
sys.path.append(r'R:\Codes\Reto\Modelos')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization, Activation, ReLU, Conv2DTranspose

# Definición de la arquitectura U-Net
def unet_model(input_shape=(112, 112, 1), n_classes=1, kernel_out=1, activation='sigmoid'):
    inputs = Input(shape=input_shape)

    conv1 = Conv2D(64, 3, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)
    conv1 = Conv2D(64, 3, padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)
    conv2 = Conv2D(128, 3, padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = ReLU()(conv3)
    conv3 = Conv2D(256, 3, padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = ReLU()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = ReLU()(conv4)
    conv4 = Conv2D(512, 3, padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = ReLU()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = ReLU()(conv5)
    conv5 = Conv2D(1024, 3, padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = ReLU()(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    up6 = concatenate([up6, drop4])
    conv6 = Conv2D(512, 3, padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = ReLU()(conv6)
    conv6 = Conv2D(512, 3, padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = ReLU()(conv6)

    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, 3, padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = ReLU()(conv7)
    conv7 = Conv2D(256, 3, padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = ReLU()(conv7)

    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, 3, padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = ReLU()(conv8)
    conv8 = Conv2D(128, 3, padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = ReLU()(conv8)

    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, 3, padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = ReLU()(conv9)
    conv9 = Conv2D(64, 3, padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = ReLU()(conv9)

    conv10 = Conv2D(n_classes, kernel_out, activation=activation)(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=conv10, name='U-Net_2')
    
    return model

model = unet_model(input_shape=(112, 112, 1), n_classes=7, kernel_out=1, activation='softmax')

In [ ]:
BATCH_SIZE = 16

# with tf.device('CPU'):
#     train_dataset = tf.data.Dataset.from_tensor_slices((train_images[:700], train_Landmarks[:700]))
#     train_dataset = train_dataset.batch(BATCH_SIZE)

#     val_dataset = tf.data.Dataset.from_tensor_slices((val_images[:300], val_Landmarks[:300]))
#     val_dataset = val_dataset.batch(BATCH_SIZE)

#     test_dataset = tf.data.Dataset.from_tensor_slices((test_images[:300], test_Landmarks[:300]))
#     test_dataset = test_dataset.batch(BATCH_SIZE)

with tf.device('CPU'):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_Landmarks))
    train_dataset = train_dataset.batch(BATCH_SIZE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_Landmarks))
    val_dataset = val_dataset.batch(BATCH_SIZE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_Landmarks))
    test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
def mean_squared_error_landmarks(y_true, y_pred):
    """
    Mean Squared Error loss function for 7 landmarks.
    Assumes y_true and y_pred are of shape (batch_size, 112, 112, 7).
    """
    mse = K.mean(K.square(y_true - y_pred), axis=-1)
    return mse

def accuracy_landmarks(y_true, y_pred):
    """
    Accuracy metric for 7 landmarks.
    Assumes y_true and y_pred are of shape (batch_size, 112, 112, 7).
    """
    # Extract the landmark values from y_true and y_pred
    y_true_landmarks = y_true[..., :7]
    y_pred_landmarks = y_pred[..., :7]

    # Calculate the element-wise absolute difference
    abs_diff = K.abs(y_true_landmarks - y_pred_landmarks)

    # Create a mask where each element is 1 if the absolute difference is less than a threshold, otherwise 0
    mask = K.cast(K.less(abs_diff, 0.5), dtype=tf.float32)

    # Calculate the accuracy for each landmark
    landmark_accuracy = K.mean(mask, axis=-1)

    # Overall accuracy is the mean accuracy across all landmarks
    overall_accuracy = K.mean(landmark_accuracy)

    return overall_accuracy

In [ ]:
import tensorflow as tf

loss_method = 'mean_squared_error'

metrics_list = ['mean_squared_error', accuracy_landmarks]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss_method, metrics=metrics_list)

KeyboardInterrupt: 

In [ ]:
# Agregamos un callback para guardar el modelo cada 5 épocas y otro para detener el entrenamiento si no hay mejora en 10 épocas
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, mode='min', monitor='val_loss'),
]

# Entrenamos el modelo
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=callbacks)

In [ ]:
evaluation = model.evaluate(test_dataset)

In [ ]:
# Save model history information to a json file
import json
import time
import matplotlib.pyplot as plt

# Generate a folder name
name_folder = 'landmark_{}'.format(time.strftime("%Y%m%d-%H%M%S"))
path = '../Pruebas/{}/'.format(name_folder)
os.mkdir(path)

# Save history to a json file
path_json = '{}history.json'.format(path)

with open(path_json, 'w') as fp:
    json.dump(history.history, fp)

# Save model
path_model = '{}model.h5'.format(path)
model.save(path_model)

# Save model summary to a txt file
path_summary = '{}summary.txt'.format(path)
with open(path_summary, 'w') as fp:
    model.summary(print_fn=lambda x: fp.write(x + '\n'))

# Save model metrics to a txt file
path_metrics = '{}metrics.txt'.format(path)
with open(path_metrics, 'w') as fp:
    fp.write('Test loss: {}\n'.format(evaluation[0]))
    fp.write('Test accuracy_landmarks: {}\n'.format(evaluation[1]))
    fp.write('Model optimizer: {}\n'.format(model.optimizer.__class__.__name__))
    fp.write('Model activation: {}\n'.format(model.layers[-1].activation.__name__))
    fp.write('Model learning rate: {}\n'.format(model.optimizer.lr.numpy()))
    fp.write('Model loss method: {}\n'.format(loss_method))

# Generate a folder inside the folder of the model for the images
path_images = '{}images/'.format(path)
os.mkdir(path_images)

# Generate a folder inside the folder of the images for plot images
path_plot_images = '{}plot_images/'.format(path_images)
os.mkdir(path_plot_images)

# Generate a folder inside the folder of the images for prediction images
path_test_images = '{}predicitons/'.format(path_images)
os.mkdir(path_test_images)

# Save prediction images
# Función para visualizar una muestra de imágenes y sus máscaras de segmentación predichas
def visualize_landmarks(images, landmarks, predictions):
    num_images = len(images)

    for i in range(num_images):  # Renamed the inner loop variable to 'j'
        # Set up the subplots
        image = images[i].reshape((112, 112))
        prediction = predictions[i].reshape((112, 112, 7)) * 255.0
        landmark = landmarks[i] * 255.0  # No need to reshape
        fig, axs = plt.subplots(2, 9, figsize=(15, 5))

        # Display the test image spanning the first two columns of both rows
        axs[0, 0].imshow(image, cmap='grey')
        axs[0, 0].axis('off')
        axs[0, 0].set_title('Test Image') 
        axs[1, 0].axis('off')

        # Combine all channels of prediction_landmarks by taking the mean
        combined_prediction = np.mean(prediction, axis=-1)

        # CObine all channels of ground_truth_landmarks by taking the mean
        combined_ground_truth = np.mean(landmark, axis=-1)

        # Display the predicted and ground truth landmarks in the first column of the second row all in one image
        axs[0, 1].imshow(combined_prediction, cmap='grey')
        axs[0, 1].set_title('Predicted')
        axs[0, 1].axis('off')
        axs[1, 1].imshow(combined_ground_truth, cmap='grey')
        axs[1, 1].set_title('Ground Truth')
        axs[1, 1].axis('off')

        # Display the predicted landmarks in the first row
        for k in range(7):  # Renamed the loop variable to 'k'
            axs[0, k+2].imshow(prediction[ :, :, k], cmap='grey')
            axs[0, k+2].axis('off')
            axs[0, k+2].set_title('Predicted ' + str(k + 1))

        # Display the ground truth landmarks in the second row
        for k in range(7):  # Renamed the loop variable to 'k'
            axs[1, k+2].imshow(landmark[ :, :, k], cmap='grey')
            axs[1, k+2].axis('off')
            axs[1, k+2].set_title('Ground Truth ' + str(k + 1))

        plt.savefig('{}{}.png'.format(path_test_images, i))


# Supongamos que tienes un conjunto de datos de prueba con imágenes y máscaras
# Puedes utilizar el modelo para obtener las máscaras predichas en el conjunto de prueba
predictions = model.predict(test_dataset)

# Elije algunas muestras aleatorias para visualizar
num_samples_to_visualize = 5
sample_indices = np.random.choice(len(test_images[:10]), num_samples_to_visualize, replace=False)

sample_images = [test_images[i] for i in sample_indices]
sample_landmarks = [test_Landmarks[i] for i in sample_indices]
sample_predictions = [predictions[i] for i in sample_indices]

# Llama a la función para visualizar las imágenes y máscaras
visualize_landmarks(sample_images, sample_landmarks, sample_predictions)

# Save plot images
plt.figure()
plt.plot(history.history['accuracy_landmarks'])
plt.plot(history.history['val_accuracy_landmarks'])
plt.title('Model accuracy_landmarks')
plt.ylabel('accuracy_landmarks')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('{}{}.png'.format(path_plot_images, 'accuracy_landmarks'))

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('{}loss.png'.format(path_plot_images))
